15 sept 2024, 15:00 Fred
Comme il manque quelques graphs et stats, j'ai pensé  le passé en python. J'ai 
commencé à écrire un script qui génère des graphs et des stats à partir des données
de la base de données. 
Run with Python 3.8.10 ENV

In [2]:
import pandas as pd

# Read the Excel file
file_path = 'DATA dSAT.xlsx'
data = pd.read_excel(file_path, sheet_name='Tidy')

# Print the DataFrame
print(data)

     Column1 Phenotype  Animal No  Stimulus Duration Distractor Condition  \
0         51        ST      377.0                 25                  SAT   
1         99        ST      377.0                 25                 dSAT   
2        147        ST      377.0                 50                  SAT   
3        219        ST      377.0                 50                 dSAT   
4        294        ST      377.0                500                  SAT   
..       ...       ...        ...                ...                  ...   
315      239        ST      478.0                 50                 dSAT   
316      334        ST      478.0                500                  SAT   
317      335        ST      478.0                500                  SAT   
318      430        ST      478.0                500                 dSAT   
319      431        ST      478.0                500                 dSAT   

         HITS    MISSES        FA         CR        OM  
0    1.000000  5.3

In [ ]:
# Create the mapping for stimulus duration and distractor condition for available columns
adjusted_stimulus_durations = {
    'HITS': [25, 25, 50, 50, 500, 500],
    'MISSES': [25, 25, 50, 50, 500, 500],
    'FA': [25, 50],
    'CR': [25, 50],
    'OM': [25, 25, 50, 50, 500, 500, 500, 500]
}

adjusted_distractor_conditions = {
    'HITS': ['SAT', 'dSAT', 'SAT', 'dSAT', 'SAT', 'dSAT'],
    'MISSES': ['SAT', 'dSAT', 'SAT', 'dSAT', 'SAT', 'dSAT'],
    'FA': ['SAT', 'dSAT'],
    'CR': ['SAT', 'dSAT'],
    'OM': ['SAT', 'dSAT', 'SAT', 'dSAT', 'SAT', 'dSAT', 'SAT', 'dSAT']
}

# Define the columns for each measure
all_measures_columns_adjusted = {
    'HITS': ['HITS', 'HITS.1', 'HITS.2', 'HITS.3', 'HITS.4', 'HITS.5'],
    'MISSES': ['MISSES', 'MISSES.1', 'MISSES.2', 'MISSES.3', 'MISSES.4', 'MISSES.5'],
    'FA': ['FA', 'FA.1'],  # Adjusted to available FA columns
    'CR': ['CR', 'CR.1'],  # Adjusted to available CR columns
    'OM': ['OM', 'OM.1', 'OM.2', 'OM.3', 'OM.4', 'OM.5', 'OM.6', 'OM.7']  # OM has additional columns
}

# Re-run the process to reorganize the data into a single tidy dataframe with more robust mapping
all_measures_long_fixed = pd.DataFrame()

for measure, columns in all_measures_columns_adjusted.items():
    # Reshape each measure to long format
    measure_long = satmerged_data[['Phenotype', 'Animal No'] + columns].melt(
        id_vars=['Phenotype', 'Animal No'], value_vars=columns, 
        var_name='Condition', value_name=measure
    )
    
    # Apply the mapping for stimulus duration and distractor condition
    measure_long['Stimulus Duration'] = measure_long['Condition'].apply(
        lambda x: adjusted_stimulus_durations[measure][int(x.split('.')[-1]) if '.' in x else 0]
    )
    measure_long['Distractor Condition'] = measure_long['Condition'].apply(
        lambda x: adjusted_distractor_conditions[measure][int(x.split('.')[-1]) if '.' in x else 0]
    )
    
    # Drop the old 'Condition' column
    measure_long = measure_long.drop(columns=['Condition'])
    
    # Merge the measures together
    if all_measures_long_fixed.empty:
        all_measures_long_fixed = measure_long
    else:
        all_measures_long_fixed = pd.merge(all_measures_long_fixed, measure_long, 
                                           on=['Phenotype', 'Animal No', 'Stimulus Duration', 'Distractor Condition'], 
                                           how='outer')

# Now you have the final tidy data in all_measures_long_fixed


In [3]:
# I'll recreate the plot and explicitly set the x-axis as categorical labels to ensure even spacing
plt.figure(figsize=(10, 6))

# Create a categorical version of the stimulus durations for even spacing
data['Stimulus Duration Label'] = data['Stimulus Duration'].map({25: '25 ms', 50: '50 ms', 500: '500 ms'})

# Lineplot for HITS, with different phenotypes and line styles for distractor conditions
sns.lineplot(
    x='Stimulus Duration Label', 
    y='HITS', 
    hue='Phenotype', 
    style='Distractor Condition', 
    markers=True, 
    dashes=[(2, 2), (1, 0)],  # Dashed for dSAT, solid for SAT
    data=data,
    palette='Set1'
)

# Add title and labels
plt.title('HITS by Stimulus Duration, Phenotype, and Distractor Condition')
plt.xlabel('Stimulus Duration')
plt.ylabel('HITS')

# Display the plot
plt.show()


NameError: name 'plt' is not defined

In [3]:
import pandas as pd
import pingouin as pg

# Assuming the tidy data is loaded in a DataFrame called `tidy_data`
# Make sure the data includes columns: 'HITS', 'Stimulus Duration', 'Distractor Condition', 'Phenotype', 'Animal No'

# Filter the data for HITS and drop any rows with missing values
tidy_hits_data = data.dropna(subset=['HITS'])

# Run the repeated measures ANOVA using pingouin
# 'Stimulus Duration' and 'Distractor Condition' are within-subject factors, 'Phenotype' is a between-subject factor
rm_anova_results = pg.rm_anova(
    dv='HITS',  # Dependent variable
    within=['Stimulus Duration', 'Distractor Condition'],  # Within-subject factors
    subject='Animal No',  # Subject identifier
    between='Phenotype',  # Between-subject factor
    data=tidy_hits_data,  # Dataframe with tidy data
    detailed=True
)

# Print the results
print(rm_anova_results)



TypeError: rm_anova() got an unexpected keyword argument 'between'